In [1]:
#!pip install requests
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install lxml


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import json
import datetime as dt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import calendar

import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words = set(stopwords.words('english'))
import pickle

In [2]:
url_base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
RSA_KEY = "FY62VLWDPNHH845BSLWHAFTVQ"
API_TOKEN = "prymaty"
location = "Kyiv"
date = "next24hours"
url_api_version = "v1"
    
url = f"{url_base_url}/{location}/{date}?unitGroup=metric&include=hours&key={RSA_KEY}&contentType=json"


headers = {
  'X-Api-Key': API_TOKEN
}

response = requests.get(url, headers=headers)

In [3]:
if response.status_code == requests.codes.ok:
    print(response.text)
else:
    print("Error:", response.status_code, response.text)

{"queryCost":1,"latitude":50.4506,"longitude":30.5243,"resolvedAddress":"Київ, Україна","address":"Kyiv","timezone":"Europe/Kiev","tzoffset":3.0,"days":[{"datetime":"2024-04-21","datetimeEpoch":1713646800,"tempmax":9.1,"tempmin":5.3,"temp":7.4,"feelslikemax":8.2,"feelslikemin":1.6,"feelslike":5.0,"dew":5.0,"humidity":85.1,"precip":27.8,"precipprob":100.0,"precipcover":83.33,"preciptype":["rain"],"snow":0.0,"snowdepth":0.0,"windgust":52.9,"windspeed":20.5,"winddir":345.3,"pressure":1005.5,"cloudcover":100.0,"visibility":11.9,"solarradiation":21.1,"solarenergy":1.8,"uvindex":1.0,"severerisk":10.0,"sunrise":"05:51:23","sunriseEpoch":1713667883,"sunset":"20:02:50","sunsetEpoch":1713718970,"moonphase":0.41,"conditions":"Rain, Overcast","description":"Cloudy skies throughout the day with a chance of rain throughout the day.","icon":"rain","stations":["remote"],"source":"comb","hours":[{"datetime":"00:00:00","datetimeEpoch":1713646800,"temp":9.1,"feelslike":8.2,"humidity":59.82,"dew":1.7,"pre

In [4]:
response_json = json.loads(response.text)
response_json

{'queryCost': 1,
 'latitude': 50.4506,
 'longitude': 30.5243,
 'resolvedAddress': 'Київ, Україна',
 'address': 'Kyiv',
 'timezone': 'Europe/Kiev',
 'tzoffset': 3.0,
 'days': [{'datetime': '2024-04-21',
   'datetimeEpoch': 1713646800,
   'tempmax': 9.1,
   'tempmin': 5.3,
   'temp': 7.4,
   'feelslikemax': 8.2,
   'feelslikemin': 1.6,
   'feelslike': 5.0,
   'dew': 5.0,
   'humidity': 85.1,
   'precip': 27.8,
   'precipprob': 100.0,
   'precipcover': 83.33,
   'preciptype': ['rain'],
   'snow': 0.0,
   'snowdepth': 0.0,
   'windgust': 52.9,
   'windspeed': 20.5,
   'winddir': 345.3,
   'pressure': 1005.5,
   'cloudcover': 100.0,
   'visibility': 11.9,
   'solarradiation': 21.1,
   'solarenergy': 1.8,
   'uvindex': 1.0,
   'severerisk': 10.0,
   'sunrise': '05:51:23',
   'sunriseEpoch': 1713667883,
   'sunset': '20:02:50',
   'sunsetEpoch': 1713718970,
   'moonphase': 0.41,
   'conditions': 'Rain, Overcast',
   'description': 'Cloudy skies throughout the day with a chance of rain through

In [5]:
response_json['days'][1]

{'datetime': '2024-04-22',
 'datetimeEpoch': 1713733200,
 'tempmax': 13.3,
 'tempmin': 5.2,
 'temp': 8.9,
 'feelslikemax': 13.3,
 'feelslikemin': 1.5,
 'feelslike': 7.2,
 'dew': 2.8,
 'humidity': 67.1,
 'precip': 2.3,
 'precipprob': 90.3,
 'precipcover': 12.5,
 'preciptype': ['rain'],
 'snow': 0.0,
 'snowdepth': 0.0,
 'windgust': 46.8,
 'windspeed': 18.4,
 'winddir': 265.1,
 'pressure': 1013.1,
 'cloudcover': 93.6,
 'visibility': 20.6,
 'solarradiation': 179.6,
 'solarenergy': 15.6,
 'uvindex': 7.0,
 'severerisk': 10.0,
 'sunrise': '05:49:24',
 'sunriseEpoch': 1713754164,
 'sunset': '20:04:25',
 'sunsetEpoch': 1713805465,
 'moonphase': 0.45,
 'conditions': 'Rain, Overcast',
 'description': 'Cloudy skies throughout the day with early morning rain.',
 'icon': 'rain',
 'stations': None,
 'source': 'fcst',
 'hours': [{'datetime': '00:00:00',
   'datetimeEpoch': 1713733200,
   'temp': 5.2,
   'feelslike': 1.5,
   'humidity': 86.31,
   'dew': 3.1,
   'precip': 1.1,
   'precipprob': 90.3,
   

In [6]:
curr_hour = dt.datetime.now().hour + 1
next_hour = 0
print(f'{curr_hour}\n{curr_hour}')

16
16


In [7]:
str(dt.datetime.today().date())

'2024-04-21'

In [8]:
response_json['days'][0]['hours'][curr_hour]['datetime']

'16:00:00'

In [9]:
result = {}
if curr_hour > 11:
    while curr_hour != 24:
        first_res = {
            'day_tempmax': response_json['days'][0]['tempmax'],
            'day_tempmin': response_json['days'][0]['tempmin'],
            'day_temp': response_json['days'][0]['temp'],
            'day_dew': response_json['days'][0]['dew'],
            'day_humidity': response_json['days'][0]['humidity'],
            'day_precip': response_json['days'][0]['precip'],
            'day_precipcover': response_json['days'][0]['precipcover'],
            'day_snow': response_json['days'][0]['snow'],
            'day_windgust': response_json['days'][0]['windgust'],
            'day_windspeed': response_json['days'][0]['windspeed'],
            'day_winddir': response_json['days'][0]['winddir'],
            'day_pressure': response_json['days'][0]['pressure'],
            'day_cloudcover': response_json['days'][0]['cloudcover'],
            'day_visibility': response_json['days'][0]['visibility'],
            'day_solarradiation': response_json['days'][0]['solarradiation'],
            'day_solarenergy': response_json['days'][0]['solarenergy'],
            'day_uvindex': response_json['days'][0]['uvindex'],
            'day_moonphase': response_json['days'][0]['moonphase'],
            'hour_temp': response_json['days'][0]['hours'][curr_hour]['temp'],
            'hour_humidity': response_json['days'][0]['hours'][curr_hour]['humidity'],
            'hour_dew': response_json['days'][0]['hours'][curr_hour]['dew'],
            'hour_precip': response_json['days'][0]['hours'][curr_hour]['precip'],
            'hour_precipprob': response_json['days'][0]['hours'][curr_hour]['precipprob'],
            'hour_snow': response_json['days'][0]['hours'][curr_hour]['snow'],
            'hour_snowdepth': response_json['days'][0]['hours'][curr_hour]['snowdepth'],
            'hour_windgust': response_json['days'][0]['hours'][curr_hour]['windgust'],
            'hour_windspeed': response_json['days'][0]['hours'][curr_hour]['windspeed'],
            'hour_winddir': response_json['days'][0]['hours'][curr_hour]['winddir'],
            'hour_pressure': response_json['days'][0]['hours'][curr_hour]['pressure'],
            'hour_visibility': response_json['days'][0]['hours'][curr_hour]['visibility'],
            'hour_cloudcover': response_json['days'][0]['hours'][curr_hour]['cloudcover'],
            'hour_solarradiation': response_json['days'][0]['hours'][curr_hour]['solarradiation'],
            'hour_uvindex': response_json['days'][0]['hours'][curr_hour]['uvindex'],
            'hour_severerisk': response_json['days'][0]['hours'][curr_hour]['severerisk'],
            'day_of_week': dt.datetime.now().day % 7,
            'date1': str(dt.datetime.today().date())
        }
        date = f"{location}, {response_json['days'][0]['datetime']}, {response_json['days'][0]['hours'][curr_hour]['datetime']}"
        curr_hour += 1
        result[date] = first_res
    while len(result) != 12:
        second_res = {
            'day_tempmax': response_json['days'][1]['tempmax'],
            'day_tempmin': response_json['days'][1]['tempmin'],
            'day_temp': response_json['days'][1]['temp'],
            'day_dew': response_json['days'][1]['dew'],
            'day_humidity': response_json['days'][1]['humidity'],
            'day_precip': response_json['days'][1]['precip'],
            'day_precipcover': response_json['days'][1]['precipcover'],
            'day_snow': response_json['days'][1]['snow'],
            'day_windgust': response_json['days'][1]['windgust'],
            'day_windspeed': response_json['days'][1]['windspeed'],
            'day_winddir': response_json['days'][1]['winddir'],
            'day_pressure': response_json['days'][1]['pressure'],
            'day_cloudcover': response_json['days'][1]['cloudcover'],
            'day_visibility': response_json['days'][1]['visibility'],
            'day_solarradiation': response_json['days'][1]['solarradiation'],
            'day_solarenergy': response_json['days'][1]['solarenergy'],
            'day_uvindex': response_json['days'][1]['uvindex'],
            'day_moonphase': response_json['days'][1]['moonphase'],
            'hour_temp': response_json['days'][1]['hours'][next_hour]['temp'],
            'hour_humidity': response_json['days'][1]['hours'][next_hour]['humidity'],
            'hour_dew': response_json['days'][1]['hours'][next_hour]['dew'],
            'hour_precip': response_json['days'][1]['hours'][next_hour]['precip'],
            'hour_precipprob': response_json['days'][1]['hours'][next_hour]['precipprob'],
            'hour_snow': response_json['days'][1]['hours'][next_hour]['snow'],
            'hour_snowdepth': response_json['days'][1]['hours'][next_hour]['snowdepth'],
            'hour_windgust': response_json['days'][1]['hours'][next_hour]['windgust'],
            'hour_windspeed': response_json['days'][1]['hours'][next_hour]['windspeed'],
            'hour_winddir': response_json['days'][1]['hours'][next_hour]['winddir'],
            'hour_pressure': response_json['days'][1]['hours'][next_hour]['pressure'],
            'hour_visibility': response_json['days'][1]['hours'][next_hour]['visibility'],
            'hour_cloudcover': response_json['days'][1]['hours'][next_hour]['cloudcover'],
            'hour_solarradiation': response_json['days'][1]['hours'][next_hour]['solarradiation'],
            'hour_uvindex': response_json['days'][1]['hours'][next_hour]['uvindex'],
            'hour_severerisk': response_json['days'][1]['hours'][next_hour]['severerisk'],
            'day_of_week': (dt.datetime.today() + dt.timedelta(days=1)).day % 7,
            'date1': str(dt.datetime.today().date())
        }
        date = f"{location}, {response_json['days'][1]['datetime']}, {response_json['days'][1]['hours'][next_hour]['datetime']}"
        next_hour += 1
        result[date] = second_res
else:
    for i in range(12):
        first_res = {
            'day_tempmax': response_json['days'][0]['tempmax'],
            'day_tempmin': response_json['days'][0]['tempmin'],
            'day_temp': response_json['days'][0]['temp'],
            'day_dew': response_json['days'][0]['dew'],
            'day_humidity': response_json['days'][0]['humidity'],
            'day_precip': response_json['days'][0]['precip'],
            'day_precipcover': response_json['days'][0]['precipcover'],
            'day_snow': response_json['days'][0]['snow'],
            'day_windgust': response_json['days'][0]['windgust'],
            'day_windspeed': response_json['days'][0]['windspeed'],
            'day_winddir': response_json['days'][0]['winddir'],
            'day_pressure': response_json['days'][0]['pressure'],
            'day_cloudcover': response_json['days'][0]['cloudcover'],
            'day_visibility': response_json['days'][0]['visibility'],
            'day_solarradiation': response_json['days'][0]['solarradiation'],
            'day_solarenergy': response_json['days'][0]['solarenergy'],
            'day_uvindex': response_json['days'][0]['uvindex'],
            'day_moonphase': response_json['days'][0]['moonphase'],
            'hour_temp': response_json['days'][0]['hours'][curr_hour]['temp'],
            'hour_humidity': response_json['days'][0]['hours'][curr_hour]['humidity'],
            'hour_dew': response_json['days'][0]['hours'][curr_hour]['dew'],
            'hour_precip': response_json['days'][0]['hours'][curr_hour]['precip'],
            'hour_precipprob': response_json['days'][0]['hours'][curr_hour]['precipprob'],
            'hour_snow': response_json['days'][0]['hours'][curr_hour]['snow'],
            'hour_snowdepth': response_json['days'][0]['hours'][curr_hour]['snowdepth'],
            'hour_windgust': response_json['days'][0]['hours'][curr_hour]['windgust'],
            'hour_windspeed': response_json['days'][0]['hours'][curr_hour]['windspeed'],
            'hour_winddir': response_json['days'][0]['hours'][curr_hour]['winddir'],
            'hour_pressure': response_json['days'][0]['hours'][curr_hour]['pressure'],
            'hour_visibility': response_json['days'][0]['hours'][curr_hour]['visibility'],
            'hour_cloudcover': response_json['days'][0]['hours'][curr_hour]['cloudcover'],
            'hour_solarradiation': response_json['days'][0]['hours'][curr_hour]['solarradiation'],
            'hour_uvindex': response_json['days'][0]['hours'][curr_hour]['uvindex'],
            'hour_severerisk': response_json['days'][0]['hours'][curr_hour]['severerisk'],
            'day_of_week': dt.today().day(),
            'date1': str(dt.datetime.today().date())
        }
        date = f"{location}, {response_json['days'][0]['datetime']}, {response_json['days'][0]['hours'][curr_hour]['datetime']}"
        result[date] = first_res
        curr_hour += 1
print(json.dumps(result, indent = 4))
# for i in range(len(result)):
#    print(result[i],'\n')


{
    "Kyiv, 2024-04-21, 16:00:00": {
        "day_tempmax": 9.1,
        "day_tempmin": 5.3,
        "day_temp": 7.4,
        "day_dew": 5.0,
        "day_humidity": 85.1,
        "day_precip": 27.8,
        "day_precipcover": 83.33,
        "day_snow": 0.0,
        "day_windgust": 52.9,
        "day_windspeed": 20.5,
        "day_winddir": 345.3,
        "day_pressure": 1005.5,
        "day_cloudcover": 100.0,
        "day_visibility": 11.9,
        "day_solarradiation": 21.1,
        "day_solarenergy": 1.8,
        "day_uvindex": 1.0,
        "day_moonphase": 0.41,
        "hour_temp": 8.3,
        "hour_humidity": 89.04,
        "hour_dew": 6.6,
        "hour_precip": 0.0,
        "hour_precipprob": 93.5,
        "hour_snow": 0.0,
        "hour_snowdepth": 0.0,
        "hour_windgust": 27.4,
        "hour_windspeed": 10.8,
        "hour_winddir": 318.0,
        "hour_pressure": 1003.0,
        "hour_visibility": 1.6,
        "hour_cloudcover": 100.0,
        "hour_solarradiation": 

In [10]:
# df = pd.read_json(result)
# df
data = json.dumps(result, indent = 4)
df = pd.read_json(data)
df

C:\Users\User\AppData\Local\Temp\ipykernel_22520\3502772875.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(data)


,"Kyiv, 2024-04-21, 16:00:00","Kyiv, 2024-04-21, 17:00:00","Kyiv, 2024-04-21, 18:00:00","Kyiv, 2024-04-21, 19:00:00","Kyiv, 2024-04-21, 20:00:00","Kyiv, 2024-04-21, 21:00:00","Kyiv, 2024-04-21, 22:00:00","Kyiv, 2024-04-21, 23:00:00","Kyiv, 2024-04-22, 00:00:00","Kyiv, 2024-04-22, 01:00:00","Kyiv, 2024-04-22, 02:00:00","Kyiv, 2024-04-22, 03:00:00"
day_tempmax,9.1,9.1,9.1,9.1,9.1,9.1,9.1,9.1,13.3,13.3,13.3,13.3
day_tempmin,5.3,5.3,5.3,5.3,5.3,5.3,5.3,5.3,5.2,5.2,5.2,5.2
day_temp,7.4,7.4,7.4,7.4,7.4,7.4,7.4,7.4,8.9,8.9,8.9,8.9
day_dew,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.8,2.8,2.8,2.8
day_humidity,85.1,85.1,85.1,85.1,85.1,85.1,85.1,85.1,67.1,67.1,67.1,67.1
day_precip,27.8,27.8,27.8,27.8,27.8,27.8,27.8,27.8,2.3,2.3,2.3,2.3
day_precipcover,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,12.5,12.5,12.5,12.5
day_snow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
day_windgust,52.9,52.9,52.9,52.9,52.9,52.9,52.9,52.9,46.8,46.8,46.8,46.8
day_windspeed,20.5,20.5,20.5,20.5,20.5,20.5,20.5,20.5,18.4,18.4,18.4,18.4


In [11]:
weather_df = df.T
weather_df

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_snow,day_windgust,day_windspeed,...,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,day_of_week,date1
"Kyiv, 2024-04-21, 16:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,10.8,318.0,1003.0,1.6,100.0,148.0,1.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 17:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,13.3,314.9,1003.0,18.8,100.0,78.0,1.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 18:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,14.8,310.3,1003.0,10.6,100.0,55.0,1.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 19:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,15.8,304.3,1003.0,2.8,100.0,5.0,0.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 20:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,16.2,308.1,1004.0,7.7,100.0,1.0,0.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 21:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,16.9,297.0,1004.0,8.6,100.0,0.0,0.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 22:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,17.6,292.8,1004.0,4.6,100.0,0.0,0.0,10.0,0,2024-04-21
"Kyiv, 2024-04-21, 23:00:00",9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,18.0,286.7,1004.0,6.5,100.0,0.0,0.0,10.0,0,2024-04-21
"Kyiv, 2024-04-22, 00:00:00",13.3,5.2,8.9,2.8,67.1,2.3,12.5,0.0,46.8,18.4,...,18.4,280.3,1005.0,8.0,100.0,0.0,0.0,10.0,1,2024-04-21
"Kyiv, 2024-04-22, 01:00:00",13.3,5.2,8.9,2.8,67.1,2.3,12.5,0.0,46.8,18.4,...,18.0,277.1,1005.0,4.2,100.0,0.0,0.0,10.0,1,2024-04-21


<h1>Get script yesterday isw</h1>

<h3>get date</h3>

In [12]:
def get_date(soup):
    date_span = soup.find('span', class_='submitted')
    date = date_span.find('span').get('content')
    date_object = dt.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S%z')
    return date_object.date()

<h3>get title</h3>

In [13]:
def get_title(soup):
    title_h1 = soup.find('h1', class_='title')
    return title_h1.text

<h3>main html</h3>

In [14]:
def get_html(soup):
    html_element = soup.find('div', class_='field field-name-body field-type-text-with-summary field-label-hidden')
    return html_element

def get_html_text(soup):
    html_element = get_html(soup)
    html_main = html_element.decode() 
    return html_main

<h3>get full url</h3>

In [15]:
def get_url(soup):
    full_url_tag = soup.find("link", rel="canonical")
    if full_url_tag:
        full_url = full_url_tag.get("href")
        return full_url

<h3>get text</h3>

In [16]:
def get_text(soup):
    html_element = get_html(soup)
    html_div = html_element.find('div', class_='field-item even')
    elements = html_div.find_all(['p', 'ul', 'ol', 'div'])
    text = ''
    for element in elements:
        if element.name == 'div' and element.find('hr'):
            break
        text += element.text
    return text

<h3>data for datetime</h3>

In [17]:
def get_data(soup):
    data = []
    data.append(get_date(soup))
    data.append(get_title(soup))
    data.append(get_url(soup))
    data.append(get_html_text(soup))
    data.append(get_text(soup))
    return data

<h3>get url for request</h3>

In [18]:
def build_url(days_to_substract):
    today = dt.datetime.today() - dt.timedelta(days=days_to_substract)
    month_name = calendar.month_name[today.month].lower()
    url = f'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-{month_name}-{today.day}-{today.year}'
    return url

<h3>get dataframe</h3>

In [19]:
def get_dataframe():
    df = pd.DataFrame(columns=['date', 'title', 'full_url', 'main_html', 'main_text'])
    days_to_substract = 0
    while True:
        days_to_substract += 1
        url = build_url(days_to_substract)
        answer = requests.get(url)
        if not answer.status_code == 200: 
            continue
        html_text = answer.text     
        soup = BeautifulSoup(html_text, 'lxml')
        df.loc[0] = get_data(soup)
        return df

In [20]:
isw_dataframe = get_dataframe()
isw_dataframe

,date,title,full_url,main_html,main_text
0,2024-04-20,"Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Russian Offensive Campaign Assessment, April 2..."


In [21]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [22]:
def lemmatize(report):
    report = report.lower()
    text = re.sub(r'\d+', '', report)
    text = re.sub(r'[^\w\s]', '', text)

    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    lemmatizer = nltk.WordNetLemmatizer()
    pos_tags = nltk.pos_tag(filtered_tokens)
    lemmatized_tokens = []
    for token, tag in pos_tags:
        wordnet_tag = get_wordnet_pos(tag)
        if wordnet_tag is None:
            lemmatized_tokens.append(token)
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(str(token), pos=wordnet_tag))
    return lemmatized_tokens

In [23]:
isw_dataframe['lemmatized_text'] = isw_dataframe['main_text'].apply(lemmatize)
isw_dataframe

,date,title,full_url,main_html,main_text,lemmatized_text
0,2024-04-20,"Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Russian Offensive Campaign Assessment, April 2...","[russian, offensive, campaign, assessment, apr..."


In [24]:
def get_str(text):
    new_text = []
    for object in text:
        new_text.append(str(object))

In [25]:
isw_dataframe['lemmatized_text'] = isw_dataframe['lemmatized_text'].apply(lambda x: ' '.join(x))
isw_dataframe

,date,title,full_url,main_html,main_text,lemmatized_text
0,2024-04-20,"Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Russian Offensive Campaign Assessment, April 2...",russian offensive campaign assessment april ri...


In [26]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(isw_dataframe['lemmatized_text'])

In [27]:
feature_names = vectorizer.get_feature_names_out()

In [28]:
tfidf_data = pd.DataFrame(X.toarray(), columns=feature_names)
tfidf_data['date1'] = str(dt.datetime.today().date())
tfidf_data.head()

,ability,able,abnormally,ac,accent,access,accommodate,accordance,achieve,across,...,yar,yarbakhmut,yarrussian,yasnobrodivka,year,yet,zaporizhia,zaporizhzhia,zorya,date1
0,0.012064,0.032171,0.004021,0.024128,0.004021,0.004021,0.004021,0.004021,0.020107,0.004021,...,0.040214,0.004021,0.004021,0.004021,0.008043,0.008043,0.004021,0.004021,0.004021,2024-04-21


In [29]:
words_data = np.load('words.npy', allow_pickle=True)
'date' in words_data

True

In [30]:
for item in words_data:
    if item not in tfidf_data.columns:
        tfidf_data[item] = 0

C:\Users\User\AppData\Local\Temp\ipykernel_22520\3564483386.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_data[item] = 0
C:\Users\User\AppData\Local\Temp\ipykernel_22520\3564483386.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_data[item] = 0
C:\Users\User\AppData\Local\Temp\ipykernel_22520\3564483386.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

In [31]:
columns_to_drop = [col for col in tfidf_data.columns if col not in words_data]
tfidf_data.drop(columns=columns_to_drop, inplace=True)

In [32]:
tfidf_data = tfidf_data.reindex(sorted(tfidf_data.columns), axis=1)

In [33]:
tfidf_data

,abandon,abandoned,abandonment,abbreviate,abduct,abduction,ability,abkhazia,able,abnormally,...,znpp,znpps,zolochiv,zolota,zolotarivka,zolote,zone,zorya,ztpp,zvanivka
0,0,0,0,0,0,0,0.012064,0,0.032171,0.004021,...,0,0,0,0,0,0,0,0.004021,0,0


In [34]:
tfidf_data['date1'] = str(dt.datetime.today().date())

C:\Users\User\AppData\Local\Temp\ipykernel_22520\2674946477.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_data['date1'] = str(dt.datetime.today().date())


In [35]:
tfidf_data.shape

(1, 6100)

In [36]:
final_df = pd.merge(weather_df, tfidf_data, on='date1')
final_df

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_snow,day_windgust,day_windspeed,...,znpp,znpps,zolochiv,zolota,zolotarivka,zolote,zone,zorya,ztpp,zvanivka
0,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
1,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
2,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
3,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
4,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
5,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
6,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
7,9.1,5.3,7.4,5.0,85.1,27.8,83.33,0.0,52.9,20.5,...,0,0,0,0,0,0,0,0.004021,0,0
8,13.3,5.2,8.9,2.8,67.1,2.3,12.5,0.0,46.8,18.4,...,0,0,0,0,0,0,0,0.004021,0,0
9,13.3,5.2,8.9,2.8,67.1,2.3,12.5,0.0,46.8,18.4,...,0,0,0,0,0,0,0,0.004021,0,0


In [37]:
final_df.drop(columns=['date1'], inplace=True)

In [38]:
final_df.shape

(12, 6134)

In [39]:
final_df['date']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
Name: date, dtype: int64

In [40]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [41]:
model

GradientBoostingClassifier(learning_rate=0.05, n_estimators=50, random_state=42)

In [51]:
predictions = model.predict_proba(final_df)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [52]:
predictions

array([[0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582],
       [0.61305418, 0.38694582]])